In [3]:
#can put if else statement into a list to make it more comprehensive
#e.g. [a_numbeer*2 for a number in some_list] ##where some_list is another list which you want to edit 

print("""Hello
everyone""") ##triple quotes let you put multiple lines in a string

Hello
everyone


In [28]:
# First a very useful function. There's nothing quite like this in itertools so I'll
# write my own.
# Note I'd normally use 'yield' here but we haven't covered that so I'll use
# an accumulator list (named res) instead.

def batches(iter_in, batch_size, include_stub=True, slide=0):
    """
    Given a list (or indeed any iterable), split the list into batches
    If the last batch is too small, we can decide whether to include this remaining stub.
    slide=0 means produce discrete chunks.
    """
    # List for the final result
    res = []
    
    if slide == 0 or slide > batch_size:
        slide = batch_size #if slide not specified then slide = batch size so are discrete slides
        
    this_batch = [] # accumulator for sub-lists
    stub = False    # is there a stub to return?
    for i in iter_in:
        this_batch.append(i)
        stub = True
        if len(this_batch) == batch_size:
            res.append(this_batch)
            stub = False
            this_batch = this_batch[slide:]
    if stub and include_stub:
        res.append(this_batch)
        
    # return everything
    return res
        
# Look it works! Note that these assertions would normally go into a separate unit test file.
# See https://docs.python.org/3/library/unittest.html for details, but note you really
# need to know about defining classes first.
assert batches([],  1) == []

assert batches("ABCDEF",  3, include_stub=True ) == [['A','B','C'],['D','E','F']]#batches of 3 specified
assert batches("ABCDEF",  3, include_stub=False) == [['A','B','C'],['D','E','F']]
assert batches("ABCDEFG", 3, include_stub=True ) == [['A','B','C'],['D','E','F'],['G']]#include stub = true keeps the extra ones 
assert batches("ABCDEFG", 3, include_stub=False) == [['A','B','C'],['D','E','F']]
assert batches(range(3),  1, include_stub=True ) == [[0],[1],[2]]
assert batches(range(3),  1, include_stub=False) == [[0],[1],[2]]

assert batches("ABCDE",  3, include_stub=True,  slide=1) == [['A','B','C'],['B','C','D'],['C','D','E']]#batches of 3 with sliding window 
assert batches("ABCDE",  3, include_stub=False, slide=1) == [['A','B','C'],['B','C','D'],['C','D','E']]
assert batches("ABC",    4, include_stub=True,  slide=1) == [['A','B','C']]
assert batches("ABC",    4, include_stub=False, slide=1) == []
assert batches(range(3), 1, include_stub=True,  slide=1) == [[0],[1],[2]]
assert batches(range(3), 1, include_stub=False, slide=1) == [[0],[1],[2]]

assert batches("ABCDE",  4, include_stub=True,  slide=3) == [['A','B','C','D'],['D','E']]
assert batches("ABCDE",  4, include_stub=False, slide=3) == [['A','B','C','D']]

In [39]:
# Now let's load the data from the files.
# Many people would say this could be better done with Pandas,
# but for now let's use regular Python (see below for Pandas).

def load_froh_2019():
    """Load old_files/Froh_2019.txt
       We only want the IID column, so return this as a set.
    """
    file_obj = open("old_files/Froh_2019.txt")
    res = set() # A set is just like a dict with no values.
    
    # If (and only if) you are splitting the line on whitespace -- .split()
    # you can dispense with the .rstrip('\n')
    headers = next(file_obj).split()  # Grab the header
    for l in file_obj:
        res.add(l.split()[headers.index('IID')])
    
    return res
    
# The file has 3047 lines so that should be 3046 individuals
all_froh_2019 = load_froh_2019()
assert len(all_froh_2019) == 3046
assert "ABS08" in all_froh_2019

def load_snp_names(k):
    """Load the AlphaPeel_files/all_chr/Deer_AlphaPeel_marker_file_chr{k}.txt
       We just want a list of all the names, so column 1.
    """
    file_obj = open(f"AlphaPeel_files/all_chr/Deer_AlphaPeel_marker_file_chr{k}.txt")
    res = []
    
    headers = next(file_obj).split()  # Grab the header
    for l in file_obj:
        res.append(l.split()[headers.index('SNP.name')])
    
    return res
    
# Chromo 10 has 561 data lines in the file.
all_snp_names_chr10 = load_snp_names("10")
assert len(all_snp_names_chr10) == 561
assert all_snp_names_chr10[0] == "cela1_red_25_1069003"
assert all_snp_names_chr10[-1] == "cela1_red_25_42782721"

def load_alphapeel_chr(k):
    """Load the chromosome matrix for chromosome k.
       Each individual has two haplotypes, so we'll get back a dict 
           {id: [hap1, hap2]}
       Where hap1 and hap2 are strings (because why not?).
    """
    file_obj = open(f"AlphaPeel_files/all_chr/AlphaPeel_out_chr{k}.called_phase.0.95")
    res = {} #returning dictionary
    
    for aline in file_obj:
        splitline = aline.split()
        res.setdefault(splitline[0],[]).append(''.join(splitline[1:]))#mushes 1010 into a string
    return res

# Chromo 10 has 12826 lines so we should get 12826 / 2 entries.
# All entries should be 561 characters long (ie. len(all_snp_names_chr10))
all_alphapeel_chr10 = load_alphapeel_chr("10")
assert len(all_alphapeel_chr10) == 12826 / 2
assert all_alphapeel_chr10['ABA14'][0].startswith('11110111')
assert all_alphapeel_chr10['ABA16'][1].endswith('10011110')
assert all([len(l) == 561 for p in all_alphapeel_chr10.values() for l in p])

In [95]:
# Pandas code. Alternative file reading with Pandas.
# Maybe we can plot the numbers in this file with seaborn??
import pandas as pd

def load_froh_2019_pandas():
    """Load old_files/Froh_2019.txt
       This uses Pandas to load the whole table. I don't use this in the code
       below, but I could do. It returns the same as the non-pandas code.
    """
    froh_table = pd.read_table("old_files/Froh_2019.txt")  #pd = pandas dataframe
    return set(froh_table['IID'])

# Should be exactly as before. Which it is.
assert len(load_froh_2019_pandas()) == 3046
assert load_froh_2019_pandas() == load_froh_2019()

In [46]:
from collections import Counter
#counter builds a dictionary to count the number of times something occurs in a list
# Function to calculate the diversity given a list of haplotypes,
# where a haplotype is just a string.
def calc_div(haplos):
    """ The formula is:
           sum( nh(nh-1) for h in distinct haplos )
           -----------------------------------
               len(haplos) * len(haplos) - 1
        Where nh is the number of times a given haplotype occurs 
        in the list 
    """
    # First thing to do is count all the haplos
    # Counter is a special type of dict which does this in one go...
    hcounts = Counter(haplos)
    # Then sum over the counts like so...
    #top = sum([ hcounts[h] * (hcounts[h] - 1) for h in haplos ])#list comprehension 
    top = sum([ n * (n - 1) for n in hcounts.values() ])
    bottom = len(haplos) * (len(haplos) - 1)
    
    return 1 - (top / bottom)


# The calc_div function should work on any list of things
assert calc_div("AAAAAA") == 0.0  # Not diverse
assert calc_div("ABCDEF") == 1.0  # Super diverse
assert calc_div("AAADEF") == 0.8  # Medium diverse

Given the data, let's generate some results. I extracted just the numbers for chr10 from
the previous results.

```bash
$ tr -d $'\r' < AlphaPeel_files/all_chr/Genomewide_hap_div_tbl.txt | \
    egrep -e '^haplo' -e '\b10$' \
    > AlphaPeel_files/chr10_hap_div_tbl.txt
```


In [99]:
# For a given chromosome matrix, get a list of chunks, where each
# chunk is a list of haplotypes (as "01010..." strings)
def get_chunks_for_chromo(chr_dict, batch_size=10, slide=0):
    """Re-format the output of load_alphapeel_chr into 
       a list of lists of haplos, where each entry in the main list is a
       batch of ten haplotype calls (columns) from the input file.
    """
    # Transform the data structure 
    all_chunks = []
    for hap_pair in chr_dict.values():
        for hap in hap_pair:
            # hap is now a string of 0s and 1s
            for n, hap_chunk in enumerate(batches( hap, 
                                                   batch_size = batch_size, 
                                                   slide = slide, 
                                                   include_stub = False )):
                if len(all_chunks) == n:
                    # If the list is too short, extend it.
                    all_chunks.append([])
                # Add this to the corresponding list
                all_chunks[n].append(''.join(hap_chunk))
                
    return all_chunks

# Again a test with some very minimal data
fake_chr_dict = { 'i1': [ '11111111',
                          '19111111' ],
                  'i2': [ '00000000',
                          '09000000' ],
                  'i3': [ '01010101',
                          '01010101' ], }
assert get_chunks_for_chromo(fake_chr_dict, 3) == [ ['111', '191', '000', '090', '010', '010'],
                                                    ['111', '111', '000', '000', '101', '101'] ]



In [98]:
# Putting all together. Just chr10 for now...
chromo = "10"

# Load up haplo matrix for this chromo (AlphaPeel_out_chr{k}.called_phase.0.95)
all_alphapeel_chrK = load_alphapeel_chr(chromo)

# Filter the inputs to just the samples from Froh_2019.txt
# Note that not all these samples are in the matrix.
all_froh_2019 = load_froh_2019()
print(f"Got {len(all_alphapeel_chr10)} samples from the matrix")
for k in list(all_alphapeel_chrK):
    if k not in all_froh_2019:
        del all_alphapeel_chrK[k]
print(f"After filtering on {len(all_froh_2019)} old samples, {len(all_alphapeel_chrK)} are left.")
print("-")

# Chunk-ify the haplotype calls
all_chunks_chrK = get_chunks_for_chromo(all_alphapeel_chrK)

# From the above shenanigans we have the all_chunks_chr10 list of lists.
# All chunks should be the same length, ie. double the size of all_alphapeel_chr10
# And for this chromosome there should be 56 chunks

# Formally check the numbers we know for chr10...
assert len(all_alphapeel_chrK) == 2939
assert all([len(chunk) == (2939 * 2) for chunk in all_chunks_chrK])
if chromo == "10":
    assert len(all_chunks_chrK) == 56
# yup!

# Load the numbers that R gave previously
ghdt_file_obj = open("AlphaPeel_files/all_chr/Genomewide_hap_div_tbl.txt")
all_rvals_chrK = [ float(ls[0]) for ls in 
                   [ line.split() for line in ghdt_file_obj ]
                   if ls[2] == chromo ]
if chromo == "10":
    # For chromo 10 we expect this many
    assert len(all_rvals_chrK) == 56

# Now get the diversity (from calc_div()) for each of those 56 chunks...
for n, chunk in enumerate(all_chunks_chrK):
    old_div_from_r = all_rvals_chrK[n]
    
    # This calculates the diversity directly...
    #div_for_chunk = calc_div(chunk)
    
    # But hang on we want the diversity excluding all the 9's
    # Could also use a regex to filter for only /^[01]+$/
    div_for_chunk = calc_div([c for c in chunk if not '9' in c])#any chunk thats got a 9 in it get discarded
    
    # Check the difference between what we got and what R said.
    difference = old_div_from_r - div_for_chunk
    print(f"{old_div_from_r} -- {div_for_chunk} -- {difference:.10f}")

Got 2939 samples from the matrix
After filtering on 3046 old samples, 2939 are left.
-
0.919218697063114 -- 0.9192186970631141 -- -0.0000000000
0.856987662458537 -- 0.8569876624585366 -- 0.0000000000
0.896464083867627 -- 0.8964640838676265 -- 0.0000000000
0.917073423365657 -- 0.9170734233656569 -- 0.0000000000
0.905695590230731 -- 0.9056955902307311 -- -0.0000000000
0.930430980098905 -- 0.9304309800989053 -- -0.0000000000
0.937756964252841 -- 0.9377569642528407 -- 0.0000000000
0.90870599656563 -- 0.9087059965656301 -- -0.0000000000
0.893955833048005 -- 0.8939558330480045 -- 0.0000000000
0.87710936239318 -- 0.87710936239318 -- 0.0000000000
0.897538404747131 -- 0.8975384047471308 -- 0.0000000000
0.907280871066946 -- 0.9072808710669462 -- -0.0000000000
0.895654727471177 -- 0.895654727471177 -- 0.0000000000
0.916601333339217 -- 0.916601333339217 -- 0.0000000000
0.907187939856013 -- 0.9071879398560134 -- -0.0000000000
0.942580933998487 -- 0.9425809339984865 -- 0.0000000000
0.913482598356507